<a href="https://colab.research.google.com/github/Covasan-Iosif/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Lab06/Solution/tema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 - CNN
## Homework 

Using the previously built pipeline (or implement a new one), implement a CNN and test it on the
CIFAR-10 dataset.
Implement the Conv2d layer by hand, using tensor operations. Your implementation needs to
be valid. Test that on the same input and weights, your implementation, and nn.Conv2d, have the
same output (use the example code below) (4 points).

In [1]:
import torch

class Handmade_conv2d_implementation(torch.nn.Module): # (input, weight, bias=None, stride=1, padding=0)
    def __init__(self, weights):
        super(Handmade_conv2d_implementation, self).__init__()
        self.weights = weights # Conv kernel

    def forward(self, x): # Forward Pass
        batch_size, input_channels, input_height, input_width = x.size()
        output_channels, input_channels, kernel_height, kernel_width = self.weights.size()

        # Initialize the output tensor (2DConvolution with stride=1 and 0 padding)
        output_height = input_height - kernel_height + 1
        output_width = input_width - kernel_width + 1
        output = torch.zeros(batch_size, output_channels, output_height, output_width) # (1, 2, 7, 8)

        # Perform the convolution
        # Iterate through dimensions
        for b in range(batch_size):
            for c_out in range(output_channels):
                for h_out in range(output_height):
                    for w_out in range(output_width):
                        # Extract the current patch from the input tensor x 
                        patch = x[b, :, h_out : h_out + kernel_height, w_out : w_out + kernel_width]
                        # Patch is a sub-tensor of dimensions (in_channels, kernel_height, kernel_width) 
                        # located at the current coordinates (h_out, w_out) in the output tensor.

                        # Perform element-wise multiplication between the patch and the weights 
                        # corresponding to the output channel
                        output[b, c_out, h_out, w_out] = (patch * self.weights[c_out]).sum()
                        # Sum the elements to get the convolution at a specific position in the output tensor

        return output

# Testing the custom Conv2D layer
inp = torch.randn(1, 3, 10, 12)  # Input image (batch_size, input_channels, input_height, input_width) 
w = torch.randn(2, 3, 4, 5)  # Conv weights (output_channels, input_channels, kernel_height, kernel_width)



In [2]:
# Instantiate the handmade Conv2D layer
custom_conv2d_layer = Handmade_conv2d_implementation(weights=w)

In [3]:
# Forward pass through the handmade Conv2D layer
out_custom = custom_conv2d_layer(inp)

In [4]:
print(out_custom.shape)

torch.Size([1, 2, 7, 8])


In [5]:
# Forward pass through the PyTorch built-in Conv2D layer
out_builtin = torch.nn.functional.conv2d(inp, w) # stride=1 and 0 padding by default

In [6]:
print(out_builtin.shape)

torch.Size([1, 2, 7, 8])


In [7]:
print((out_builtin - out_custom).abs().max())

tensor(7.6294e-06)


In [8]:
tolerance = 1e-5

if torch.allclose(out_custom, out_builtin, atol=tolerance):
    print("Cele două layere produc aproximativ același rezultat")
else:
    print("Cele două layere nu produc același rezultat.")

Cele două layere produc aproximativ același rezultat
